<a id='top'></a>
<img style="float: center;" src='https://github.com/STScI-MIRI/MRS-ExampleNB/raw/main/assets/banner1.png' alt="stsci_logo" width="1000px"/>

# NIRSpec MOS MSA Metafile

**Authors**: Dan Coe (dcoe@stsci.edu) and Kayli Glidic (kglidic@stsci.edu) with contributions from others on the STScI NIRSpec team. <br>
**Created On**: June, 2024 ([JWebbinar 33](https://www.stsci.edu/jwst/science-execution/jwebbinars.html#h3-dc1c3cd6-f1aa-45c4-b73f-e93cc96ff4a7:~:text=33%20%2D%20JWST%20NIRSpec%20MOS%20Data%20Reduction))<br>
**Updated On**: November, 2025. <br>

**Purpose**:<br> The primary goal of this notebook is to explore the Micro-Shutter Assembly (MSA) metadata file, an essential file for calibrating Near Infrared Spectrograph (NIRSpec) Multi-Object Spectroscopy (MOS) observations with the *James Webb Space Telescope* (JWST) Calibration Pipeline.

**[Data](#data)**:<br>
This notebook is set up to use the MSA metadata file from [ERO 2736](https://www.stsci.edu/cgi-bin/get-proposal-info?observatory=JWST&id=2736) observations of [SMACS0723](https://webbtelescope.org/contents/news-releases/2022/news-2022-035).

---

## Table of Contents
* [1. Introduction](#intro)
* [2. Import Library](#imports)
* [3. Helper Functions](#funcs)
* [4. Download the MSA Metafile](#data)
* [5. Load the MSA Metafile and Inspect Contents](#content)
    * [5.1 Source Table](#source_table)
    * [5.2 Shutter Table](#shutter_table)
    * [5.3 Shutter Image](#shutter_image)
        * [5.3.1 Shutter Image from MSA Metadata File](#image_meta)
        * [5.3.2 Shutter Image from Shutter Table](#image_table)
        * [5.3.3 Shutter Image from APT](#image_apt)
* [6. Modifying the MSA Metadata File](#modify)
* [7. Link to Files in Preparation for Pipeline](#pipeline)





























---
  
## 1. Introduction <a id='intro'></a>


The [MSA metadata file](https://jwst-pipeline.readthedocs.io/en/latest/jwst/data_products/msa_metadata.html) records information about NIRSpec [MOS](https://jwst-docs.stsci.edu/jwst-near-infrared-spectrograph/nirspec-observing-modes/nirspec-multi-object-spectroscopy) [MSA](https://jwst-docs.stsci.edu/jwst-near-infrared-spectrograph/nirspec-instrumentation/nirspec-micro-shutter-assembly) configurations as planned by [APT](https://jwst-docs.stsci.edu/jwst-astronomer-s-proposal-tool-overview)'s [MSA Planning Tool (MPT)](https://jwst-docs.stsci.edu/jwst-near-infrared-spectrograph/nirspec-apt-templates/nirspec-multi-object-spectroscopy-apt-template/nirspec-msa-planning-tool-mpt). This file details which shutters were planned to be opened, how the shutters were configured into slitlets, and which targets were assigned to each slitlet. The JWST *[`calwebb_spec2`](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_spec2.html)* pipeline uses the MSA metadata file to extract spectra from sources. The overall structure of the MSA FITS file is as follows:

* `SHUTTER_IMAGE` (2D array): A map showing which MSA shutters were open or closed during the observation.
* `SHUTTER_INFO` (table): Detailed information for each shutter, including whether it is part of a longer slit (slitlet), if it contains a science target or empty background, etc.
* `SOURCE_INFO` (table): A catalog of science targets plus fake targets in empty slits.


In the remainder of this notebook, we explore the contents of the MSA metadata file in more detail. We also demonstrate how to edit the file for:

* a subset of sources,
* science vs. background shutters, and
* point source vs. extended targets (used by the pathloss correction),

Finally, we also show how to view the shutter image map.






---

## 2. Import Library <a id='imports'></a>


In [ ]:
# Import libraries.

# General imports.
import os
import csv
import copy
import requests
import numpy as np
from glob import glob

# Plotting imports.
import matplotlib.pyplot as plt
import matplotlib as mpl
#%matplotlib notebook  # interactive plots
%matplotlib inline

# Astropy/query imports.
import astroquery
from astropy.io import fits
from astropy.table import Table
from astroquery.mast import Observations
print('astroquery version', astroquery.__version__)

---

## 3. Helper Functions <a id='funcs'></a>

These helper functions perform basic tasks like checking values, filtering data, and managing files.

In [ ]:
def even(x):
    """
    Even or odd number.

    Parameters
    ----------
    x : int
        Integer to be tested.
    
    Returns
    -------
    value : int
        1 = even; 0 = odd
    """
    return 1 - x % 2

def single_value(x):
    """
    Determine whether the input is a single integer or float.
    True = one number; False = multiple numbers (list / tuple / array / set).

    Parameters
    ----------
    x : int, float, list, tup, etc

    Returns
    -------
    value : bool
        True if x is an integer or float. Otherwise false.
    """
    return isinstance(x, (int, float))

def filter_table(full_table, **kwargs):
    """
    Filters an Astropy Table based an arbitrary number of input column-value pairs.
    Each value can be either a single value or a list (or tuple, array, or set).
    Example:
    select_shutter_table = filter_table(shutter_table, msa_metadata_id=1, dither_point_index=1, source_id=[6355,5144])

    Parameters
    ----------
    full_table : astropy.table.Table
        Table to be filtered.

    Returns
    -------
    filtered_table : astropy.table.Table
        Table containing only requested columns/values.
    """
    filtered_table = full_table
    for column, value in kwargs.items():
        if single_value(value):
            filtered_table = filtered_table[filtered_table[column] == value]
        else: # list
            filtered_table = filtered_table[[(item in value) for item in filtered_table[column]]]
    return filtered_table


# Create links in output_dir directory to source files.
def link_to_files(source_files,
                  output_dir):
    """
    Create symlinks to the given source_files in the output_dir directory.

    Parameters
    ----------
    source_files : list
        List of filenames.
    output_dir : str
        Directory in which to create symlinks.
    """
    for source_file in source_files:
        link = os.path.join(output_dir, os.path.basename(source_file))
        print(link, '->', source_file)
        if not os.path.exists(link):
            os.symlink(os.path.abspath(source_file), link)

Below, we define a couple of functions to download data products from MAST.

In [ ]:
# Helper function to download JWST files from MAST.
def download_jwst_files(filenames,
                        download_dir,
                        mast_dir='mast:jwst/product'):
    """
    Helper function to download JWST files from MAST.

    Parameters:
    ----------
    filenames: list of str
        List of filenames to download.
    download_dir: str
        Directory where the files will be downloaded.
    mast_dir: str
        MAST directory containing JWST products.

    Returns:
    -------
    downloaded_files: list of str
        List of downloaded file paths.
    """
    # Download data.
    downloaded_files = []
    os.makedirs(download_dir, exist_ok=True)
    for filename in filenames:
        filename = os.path.basename(filename)
        mast_path = os.path.join(mast_dir, filename)
        local_path = os.path.join(download_dir, filename)
        if os.path.exists(local_path):
            print(local_path, 'EXISTS')
        else:
            # Can let this command check if local file exists.
            # However, it will delete it if it's there
            # and the wrong size (e.g., reprocessed).
            Observations.download_file(mast_path, local_path=local_path)
        downloaded_files.append(local_path)

    return downloaded_files


def download_file(url,
                  download_dir=''):
    """
    Simple script to download small files from MAST.

    Parameters
    ----------
    url : str
        URL to a specific file in MAST.
    download_dir : str
        Directory into which the file will be downloaded.
        
    Returns
    -------
    filename : str
        Name of downloaded file.
    """
    filename = os.path.basename(url)
    if download_dir:
        if os.path.isdir(download_dir):
            filename = os.path.join(download_dir, filename)
    if os.path.exists(filename):
        print(filename, 'EXISTS')
    else:
        print('DOWNLOADING', filename)
        r = requests.get(url, allow_redirects=True)
        open(filename, 'wb').write(r.content)
    return filename

---

## 4. Download the MSA Metafile <a id='data'></a>

The MSA metadata files are available on the [Mikulski Archive for Space Telescopes (MAST)](https://archive.stsci.edu/missions-and-data/jwst) archive, and each NIRSpec MOS data product includes the associated MSA metadata filename in its header under the keyword `MSAMETFL`. In the next few cells, we download an `S2D` product for a given `source_id` and extract the name of the associated MSA metafile from its metadata.

**Additional Note**: Although we do not run the pipeline in this notebook, it's important to know that *`calwebb_spec2`* expects the MSA metadata file to be located in the same directory as the input data files. To meet this requirement, we save the file in a new directory *using the same filename* and link to the data files from that directory. The pipeline can then be run from that directory, and the headers will still point to the MSA metadata file. Alternatively, if the updated MSA metadata file were given a new filename, then you must also edit the data file headers to point to the new MSA metadata file.






In [ ]:
# If you know the filename, you can skip this section.
#msa_metafile = 'jw02736007001_01_msa.fits'

In [ ]:
# Define data directory.
data_dir = 'data'
os.makedirs(data_dir, exist_ok=True)

In [ ]:
# Download S2D file for a given source_id.
source_id = 6355  # z = 7.665
data_file = 'jw02736-o007_s%09d_nirspec_f290lp-g395m_s2d.fits' % source_id
data_file = download_jwst_files([data_file], data_dir)[0]
data_file

In [ ]:
# Get the MSA metadata file name from the S2D header.
msa_metafile_name = fits.getval(data_file, 'MSAMETFL')
msa_metafile_name

In [ ]:
# Download the MSA metafile.
msa_metafile = download_jwst_files([msa_metafile_name], data_dir)[0]
msa_metafile

---

## 5. Load the MSA Metafile and Inspect Contents <a id='content'></a>

Now, we will load the MSA metadata file we downloaded and inspect some of the contents.

In [ ]:
# Load MSA metafile and inspect contents.
msa_hdu_list = fits.open(msa_metafile)
msa_hdu_list.info()

In [ ]:
# Load these tables; we'll inspect and edit them below.
source_table = Table(msa_hdu_list['SOURCE_INFO'].data)
shutter_table = Table(msa_hdu_list['SHUTTER_INFO'].data)

In [ ]:
# Print primary header.
msa_hdu_list['PRIMARY'].header

In [ ]:
# Print the observation date.
msa_hdu_list['PRIMARY'].header['DATE']

In [ ]:
# The datamodel shows the same info in a longer format.
#from jwst import datamodels
#msa_model = datamodels.open(msa_metafile)
#msa_model.info(max_rows=99999)

---

### 5.1 Source Table <a id='source_table'></a>

The `SOURCE_INFO` table contains information on each source in the input catalog.  
"Virtual" sources in empty slits are assigned negative numbers, in sequence.  
The [source type](https://jwst-pipeline.readthedocs.io/en/latest/jwst/srctype/description.html) 
is also encoded:
* 0.0 <= `stellarity` <= 0.75: extended
* otherwise: point source

| SOURCE_INFO | Description |
|:-:|:-|
| `PROGRAM` | JWST program ID |   
| `SOURCE_ID` | Unique integer identifier: positive (real sources) or negative (virtual sources) |   
| `SOURCE_NAME` | Typically a combination of the first two columns |
| `ALIAS` | Either the source ID (real sources) or the RA, Dec (virtual sources) |   
| `RA`/`DEC` | Catalog source coordinates, in decimal degrees |   
| `PREIMAGE_ID` | Name of NIRCam pre-imaging mosaic used to determine the source catalog, if it exists |   
| `STELLARITY` | Float ranging from 0 (fully extended) to 1 (point source). <br> The pipeline treats 0 <= stellarity <= 0.75 as extended for path loss corrections. |   
[More Info ...](https://jwst-docs.stsci.edu/jwst-calibration-pipeline-caveats/jwst-nirspec-mos-pipeline-caveats#JWSTNIRSpecMOSPipelineCaveats-msa_metafileMetadataforsourceandslitletinformation)

In [ ]:
# Print out the source table.
source_table

---

### 5.2 Shutter Table <a id='shutter_table'></a>



The `SHUTTER_INFO` table specifies all the slitlets having one or more open shutters based on the MSA configuration for that observation.  

| SHUTTER_INFO | Description |
|:-:|:-|
| `SLITLET_ID` | Integer number representing each slitlet of one or more open shutters specified in the MSA configuration |  
| `MSA_METADATA_ID` | Integer number corresponding to a particular MSA configuration / MPT plan <br> (one metafile may contain multiple MSA configurations) |
| `SHUTTER_QUADRANT` | MSA quadrant (1, 2, 3, 4) |
| `SHUTTER_ROW` | Row number decreases in the dispersion direction (transposed with respect to science data) |
| `SHUTTER_COLUMN` | Column number increases in the spatial cross-dispersion direction |
| `SOURCE_ID` | Unique integer ID for each source in each slitlet, used for matching to the SOURCE_INFO table |
| `BACKGROUND` | Boolean indicating whether the shutter is open to background (Y) or contains a known source (N) (for a given nod exposure if the observation includes nodding) | 
| `SHUTTER_STATE` | Generally, this will always be OPEN, unless a long slit was used |
| `ESTIMATED_SOURCE_IN_SHUTTER_X/Y` | The position of the source within the shutter in relative units (where 0,0 is the bottom left corner and 0.5,0.5 is the center), as planned in MPT |
| `DITHER_POINT_INDEX` | Integer specifying the index of the nod sequence; matches with the data primary header keyword PATT_NUM | 
| `PRIMARY_SOURCE` | Boolean indicating whether the shutter contains the science source |
[More Info ...](https://jwst-docs.stsci.edu/jwst-calibration-pipeline-caveats/jwst-nirspec-mos-pipeline-caveats#JWSTNIRSpecMOSPipelineCaveats-msa_metafileMetadataforsourceandslitletinformation)

In [ ]:
# Print out the shutter table.
shutter_table

In [ ]:
# Multiple MSA configurations are often defined in a single file.
msa_metadata_ids = list(set(shutter_table['msa_metadata_id']))
msa_metadata_ids

In [ ]:
# MSA metadata ids and slitlet ids are all numbered sequentially, picking up where the other left off.
# Here, after the first metadata id 1, the slitlets are numbered 2 – 75, followed by the next metadata id 76.
slitlet_ids = np.sort(list(set(shutter_table['slitlet_id'])))
slitlet_ids

In [ ]:
# As an example, filter the shutter table for a specific MSA metadata id and dither point index.
shutter_table_dither1 = filter_table(shutter_table, msa_metadata_id=1, dither_point_index=1)
shutter_table_dither1

---

### 5.3 Shutter Image <a id='shutter_image'></a>

In this section, we will translate shutter (column, row) coordinates from the MSA metafile to the observed coordinate system as shown in APT.

The MSA coordinate system is shown on [JDox](https://jwst-docs.stsci.edu/jwst-near-infrared-spectrograph/nirspec-instrumentation/nirspec-micro-shutter-assembly).
Note for each quadrant, the origin is at top right, and numbers increase to the bottom left corner.

The JWST pipeline generates an [MSA metafile](https://jwst-pipeline.readthedocs.io/en/latest/jwst/data_products/msa_metadata.html)
that uses a similar coordinate system in the `SHUTTER_INFO` *except* that rows and columns are swapped:

* columns increment in the spatial direction
* row numbers decrease in the dispersion direction
    
The MSA metafile `SHUTTER_IMAGE` combines all 4 quadrants into one 2D array:  

* numbers are continuous without gaps
    * the origin (1, 1) is at top right
    * (730, 342) is at bottom left
    
...but only after shuffling them:  
* quadrants are reordered as shown below:

NIRSpec MSA quadrants:  
Q3 Q1  
Q4 Q2  

MSA metafile coordinates:  
Q3 Q4  
Q2 Q1

---

#### 5.3.1 Shutter Image from MSA Metadata File <a id='image_meta'></a>

In [ ]:
# Read in the shutter image and transpose it (swap rows and columns).
shutter_image = msa_hdu_list['SHUTTER_IMAGE'].data[:].T

ny, nx = shutter_image.shape

# Define the quadrants.
Q4 = shutter_image[:ny//2, :nx//2]
Q3 = shutter_image[:ny//2, nx//2:]
Q1 = shutter_image[ny//2:, :nx//2]
Q2 = shutter_image[ny//2:, nx//2:]

# Shuffle the MSA quadrants so that (730, 342) is at bottom left.
msa_image = np.vstack((np.hstack((Q1, Q3)), np.hstack((Q2, Q4))))
print(f'msa_image shape is (ny, nx): {msa_image.shape}')

In [ ]:
fig, ax = plt.subplots(figsize=(12,10))
cmap = copy.deepcopy(mpl.colormaps['turbo'])
cmap.colors[0] = 1, 1, 1
extent = 0.5, nx+0.5, ny+0.5, 0.5  # origin=(1,1); range = 1..nx, 1..ny with +/-0.5 boundary on either side
im = plt.imshow(msa_image, aspect=nx/ny, interpolation='nearest', cmap=cmap, extent=extent)
plt.xlim(nx+1,0)
plt.ylim(ny+1,0)
plt.xlabel('shutter row (full MSA)')
plt.ylabel('shutter column (full MSA)')
ax.yaxis.tick_right()
ax.yaxis.set_label_position("right")
ax.xaxis.tick_top()
ax.xaxis.set_label_position("top")
cbar = fig.colorbar(im, ax=ax, shrink=1, aspect=20, pad=0.02, location='left')
cbar.set_label('SLITLET_ID')

---

#### 5.3.2 Shutter Image from Shutter Table <a id='image_table'></a>

In [ ]:
# Extract quadrant, slitlet_id, row, and column from the filtered shutter table.
quadrant = shutter_table_dither1['shutter_quadrant']
slitlet = shutter_table_dither1['slitlet_id']

# Row, column resets within each quadrant.
row_in_quadrant = shutter_table_dither1['shutter_row']
column_in_quadrant = shutter_table_dither1['shutter_column']

# Row, column on full msa grid.
msa_row = row_in_quadrant + (quadrant > 2) * nx//2
msa_column = column_in_quadrant + even(quadrant) * ny//2

In [ ]:
# Plot the shutter image from the shutter table.
fig, ax = plt.subplots(figsize=(12, 10))
plt.scatter(msa_row, msa_column, c=slitlet, marker='+', cmap=cmap, clim=(0, np.max(slitlet)))
plt.xlim(nx, 0)
plt.ylim(ny, 0)
plt.xlabel('Shutter Row (Full MSA)')
plt.ylabel('Shutter Column (Full MSA)')
ax.yaxis.tick_right()
ax.yaxis.set_label_position("right")
ax.xaxis.tick_top()
ax.xaxis.set_label_position("top")
ax.set_aspect(nx/ny)
cbar = fig.colorbar(im, ax=ax, shrink=1, aspect=20, pad=0.02, location='left')
cbar.set_label('SLITLET_ID')

---

#### 5.3.3 Shutter Image from APT <a id='image_apt'></a>

From APT –> MPT –> Plans –> Pointings –> Export Config.  
This returns a CSV file with a 2D map of all shutters for that MSA configuration:
* 1 = closed
* 0 = open

In [ ]:
# Download the example MSA configuration file from APT.
msa_config_file = 'https://raw.githubusercontent.com/dancoe/NIRSpec_MOS_JWebbinar/main/data/2736.p1c1-2e1n1.csv'
msa_config_file = download_file(msa_config_file, data_dir)
msa_config_file

In [ ]:
with open(msa_config_file, 'r') as f:
    reader = csv.reader(f)
    msa_config_data = list(reader)
    
msa_config_data = np.array(msa_config_data[1:])  # remove first line header
msa_config_data = msa_config_data.astype(int).T

In [ ]:
# Plot the shutter image from APT.
fig, ax = plt.subplots(figsize=(12, 10))
cmap = copy.deepcopy(mpl.colormaps['turbo'])
cmap.colors[0] = 1, 1, 1
extent = 0.5, nx+0.5, ny+0.5, 0.5  # origin=(1,1); range = 1..nx, 1..ny with +/-0.5 boundary on either side
im = plt.imshow(1 - msa_config_data, aspect=nx/ny, interpolation='nearest', extent=extent, cmap=cmap)
plt.xlim(nx+1, 0)
plt.ylim(ny+1, 0)
plt.xlabel('Shutter Row (Full MSA)')
plt.ylabel('Shutter Column (Full MSA)')
ax.yaxis.tick_right()
ax.yaxis.set_label_position("right")
ax.xaxis.tick_top()
ax.xaxis.set_label_position("top")

---

## 6. Modifying the MSA Metadata File <a id='modify'></a>


In this section, we demonstrate several types of modifications that can be made to the MSA metadata file, including:

* Trimming the metadata to a subset of objects (sources)
* Setting a source to be either point-like or extended
* Specifying background shutters

Finally, we will show how to save the modified MSA metadata file.



In the cell below, we trim the MSA metafile to a subset of objects.

In [ ]:
#select_source_ids = -8, -9, -11, -12, -13, -16, -26, -27, -28, -29, -38, -41, -43, -57, -58, -59  # empty? backgrounds
select_source_ids = 6355, 5144, 4590, 10612, 8140, 9922  # Featured high-z galaxies.

select_source_table = filter_table(source_table,  source_id=select_source_ids)
select_shutter_table = filter_table(shutter_table, source_id=select_source_ids)

# See that only the sources of interest are in the table.
select_source_table


Next, we look at how to set a source to a point or an extended source. The Path loss correction step in the pipeline will use this information.

In [ ]:
# Here we'll simply set them all to point source.
select_source_table['stellarity'] = 1  # 0 = extended; 1 = point source

# Check that the stellarity column has been updated.
select_source_table

Lastly, we will demonstrate how to set shutters to be identified as backgrounds. The pipeline will use this information to perform background subtraction. We could do this, but we won't, so we'll comment out these lines.

In [ ]:
# Master Background.
#select_shutter_table['background']     = 'Y'
#select_shutter_table['primary_source'] = 'N'

Print out the modified source and shutter tables.

In [ ]:
select_source_table

In [ ]:
# Show contents but just for the first config and dither.
filter_table(select_shutter_table, msa_metadata_id=1, dither_point_index=1)

Save the modified MSA metafile with the same filename as the original, but in a new directory.

In [ ]:
# Create the new output directory if it does not yet exist.
output_dir = 'reprocess_subset'
os.makedirs(output_dir, exist_ok=True)

# Add the new directory to the output filename.
output_msa_metafile = os.path.join(output_dir, msa_metafile_name)
output_msa_metafile

In [ ]:
msa_hdu_list.info()

In [ ]:
# Convert tables to HDUs and add to the HDU List.
msa_hdu_list['SHUTTER_INFO'] = fits.table_to_hdu(select_shutter_table)
msa_hdu_list['SOURCE_INFO'] = fits.table_to_hdu(select_source_table)

msa_hdu_list[2].name = 'SHUTTER_INFO'
msa_hdu_list[3].name = 'SOURCE_INFO'

msa_hdu_list.info()

In [ ]:
print('SAVING', output_msa_metafile)
msa_hdu_list.writeto(output_msa_metafile, overwrite=True)
#msa_hdu_list.close()  # let's keep this open to use below.

---

## 7. Link to Files in Preparation for Pipeline <a id='pipeline'></a>

We won't run the pipeline in this notebook, but this shows you how to prepare.  
When you run the pipeline on these linked files in this new directory, it will find the updated metafile there.

In [ ]:
rate_files = sorted(glob(os.path.join(data_dir, '*_rate.fits*')))
link_to_files(rate_files, output_dir)

In [ ]:
asn_files = sorted(glob(os.path.join(data_dir, '*_asn.json')))
link_to_files(asn_files, output_dir)



---
[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/>